In [ ]:
import numpy as np
import matplotlib as mpl
import pylab
from libtiff import TIFFfile
from libtiff import TIFF
import os
from scipy import interpolate
import cPickle

In [ ]:
LineArraysFile=open("Pkls/LineArrays.pkl")
LineArrays=cPickle.load(LineArraysFile)

In [ ]:
def AssembleLines(LineArraysIn):
    LensAll=[]
    AvesAll=[]
    for i in LineArraysIn:
        LensAll.append(len(i[0]))
        AvesAll.append(np.mean(i))
    YSize=len(LineArraysIn[0])
    BGVal=np.mean(AvesAll)
    MaxLen=max(LensAll)
    FullArray=np.ndarray(shape=(len(LineArraysIn)*YSize,MaxLen))
    for i in range(0,len(LineArraysIn)):
        for a in range(0,len(LineArraysIn[i][0])):
            for j in range(0,YSize):
                FullArray[int(i*YSize+j),a]=LineArraysIn[i][int(YSize-j-1),a]
        for a in range(len(LineArraysIn[i][0]),max(LensAll)):
            for j in range(0,YSize):
                FullArray[i*YSize+j,a]=BGVal
    return FullArray


In [ ]:
RawImage=AssembleLines(LineArrays)
pylab.imshow(RawImage,cmap='binary')
pylab.imsave("Plots/Flattened.png",RawImage)

In [ ]:
def ExtractLocalBGRefAndOffset(LineArray):
    BGRefs=[]
    Offsets=[]
    
    YSize=len(LineArray)
    ArrayLength=len(LineArray[0])

    for a in range(0,ArrayLength):
            #Brightness of lowest 20%
            BGRefs.append(np.mean(sorted(LineArray[:,a])[0:int(YSize*0.2)]))
            #Median Y pos
            Offsets.append((np.mean(LineArray[:,a]*range(0,YSize))/np.mean(LineArray[:,a]))-YSize/2.)
    vars=range(0,ArrayLength)
    pylab.plot(BGRefs,label="Extracted",color='DarkRed')
    pylab.xlabel("Horizontal Coordinate")
    pylab.ylabel("Background Reference")
    polyBG=np.polyfit(vars,BGRefs,8)
    BGPoints=np.poly1d(polyBG)(vars)
    pylab.plot(vars,BGPoints,color='red',label='Smooth Fit',linewidth=2)
    pylab.legend(loc='upper left')
    pylab.show()
    pylab.plot(Offsets,color='DarkBlue')
    polyOffsets=np.polyfit(vars,Offsets,8)
    OffsetPoints=np.poly1d(polyOffsets)(vars)
    pylab.plot(vars,OffsetPoints,color='blue',linewidth=2)
    pylab.xlabel("Horizontal Coordinate")
    pylab.ylabel("Vertical Offset")
    pylab.legend(loc='upper left')



    pylab.show()    
    return [BGPoints,OffsetPoints]

In [ ]:
def RemoveBGAndOffset(LineArray):
    ReturnArray=np.zeros_like(LineArray)
    BGPoints,OffsetPoints=ExtractLocalBGRefAndOffset(LineArray)
    YSize=len(LineArray)
    ArrayLength=len(LineArray[0])

    for a in range(0,ArrayLength):
        bgval=BGPoints[a]
        for j in range(0,YSize):
            jindex=(j+int(OffsetPoints[a]))%YSize

            ReturnArray[j,a]=LineArray[jindex,a]-bgval
    return ReturnArray

In [ ]:
def Trim(LineArray,Lower,Upper):
    SubArray=LineArray[Lower:Upper,:] 
    return SubArray

In [ ]:
TrimmedBGSubtractedLineArrays=[]
BGSubtractedLineArrays=[]
for i in LineArrays:
    BGSubtractedLineArrays.append(RemoveBGAndOffset(i))
    TrimmedBGSubtractedLineArrays.append(Trim(BGSubtractedLineArrays[-1],20,250))

In [ ]:
BGSubImage=AssembleLines(BGSubtractedLineArrays)
pylab.imshow(BGSubImage,cmap='binary')
pylab.imsave("Plots/BGSubtracted.png",BGSubImage,cmap='binary')

In [ ]:
LineArraysFile=open("Pkls/LineArraysBGSub.pkl",'wb')
cPickle.dump(BGSubtractedLineArrays,LineArraysFile)
LineArraysFile.close()

In [ ]:
TrimmedBGSubImage=AssembleLines(TrimmedBGSubtractedLineArrays)
pylab.imshow(TrimmedBGSubImage,cmap='binary')
pylab.imsave("Plots/TrimmedBGSubtracted.png",TrimmedBGSubImage,cmap='binary')

In [ ]:
TrimmedLineArraysFile=open("Pkls/TrimmedLineArraysBGSub.pkl",'wb')
cPickle.dump(TrimmedBGSubtractedLineArrays,TrimmedLineArraysFile)
TrimmedLineArraysFile.close()

In [ ]:
import scipy.ndimage as ndimage

In [ ]:
Kernel=10
for i in range(0,len(BGSubtractedLineArrays)):
    img = ndimage.gaussian_filter(BGSubtractedLineArrays[i], sigma=(Kernel), order=0)
    pylab.imshow(img[::-1],cmap='binary',clim=(-20,150))
    
    pylab.show()
    pylab.imsave("BlurredLines/Blurred"+str(Kernel)+"_"+str(i)+".png",img)

In [ ]:
img = ndimage.gaussian_filter(BGSubImage, sigma=(15), order=0)
pylab.imshow(img,cmap='binary')
pylab.imsave("Plots/Blur15.png",img,cmap='binary')
pylab.savefig("Plots/Blur15_WithAxes.png",dpi=500)

In [ ]:
250-170